# 1 Task: Foundations

Solve a bin packing task in decision/evalution/search variants.

In [4]:
from typing import List

# Leave the whole “solve_bp_decision” function intact
def solve_bp_decision(items: List[float], n_bins: int) -> bool:
    def able_to_pack(items: List[float], bin_capacities: List[float]) -> bool:
        return items == [] or any( 
            able_to_pack( 
                items[:-1], 
                bin_capacities[:i] + [capacity - items[-1]] + bin_capacities[(i + 1):] 
            ) 
            for i, capacity in enumerate(bin_capacities) if capacity >= items[-1] 
        )

    return able_to_pack( sorted(items), [1.0] * n_bins )

In [45]:
def solve_bp_evaluation(items: List[float]) -> int:
    # bin search
    left = 1; right = len(items) 
    
    while left != right:
        
        curr_idx = left + round((right - left) / 2)
                
        if solve_bp_decision(items, curr_idx):
            right = curr_idx
        else:
            left  = curr_idx + 1 
    
    return right

def merge( packed: List, optimal: int ):
    
    get_weight = lambda x: sum(map(lambda y: y[1], x))
    to_items = lambda packed: [get_weight(x) for x in packed] 
    
    for i in range(len(packed)):
        for j in range(i+1, len(packed)):

            if get_weight( packed[i] ) + get_weight( packed[j] ) <= 1:

                temp = packed.copy()
                temp[i] = temp[i].union( temp[j] )
                temp.pop(j)

                if solve_bp_evaluation( to_items(temp) ) == optimal:
                    packed[i] = packed[i].union( packed[j] )
                    packed.pop(j)
                    return

def solve_bp_search(items: List[float]) -> List[int]:
    
    packed = [{(i, x)} for i, x in enumerate(items)]
    optimal = solve_bp_evaluation(items)
    
    while len(packed) != optimal:
        merge(packed, optimal)
    
    result = [0] * len(items)
    
    for i, bin in enumerate(packed):
        for el in bin:
            result[el[0]] = i + 1
            
    return result

In [43]:
a1 = [1.0]
a2 = [0.19, 0.19, 0.19, 0.19, 0.19]
a3 = [0.09, 0.19, 0.49, 0.29, 0.39, 0.49]
a4 = [0.79, 0.09, 0.39, 0.69]
a5 = [0.29, 0.29, 0.29, 0.29, 0.29, 0.29, 0.39, 0.39, 0.39]

In [85]:
solve_bp_search(a5)

[1, 1, 2, 2, 3, 3, 1, 2, 3]

# 2 Task: Linear programming

In [ ]:
from scipy.optimize import linprog
import numpy as np
from collections import defaultdict

def read_data():
    
    n_cols, n_rows = map(int, input().split())
    
    w = np.zeros(n_rows)
    matr = np.zeros([n_cols, n_rows])
    
    for i in range(n_rows):
        new = list(map(int, input().split()))
        w[i] = new[0]
        for j in new[1:]:
            matr[j, i] = 1
    
    return w, matr

# Solve lin prog problem

w, matr = read_data()

res = linprog(c=w, A_ub=-matr, b_ub=-np.ones(matr.shape[0]), bounds=[0,1])

# Construct data strucutes for the next step

col_dict = defaultdict(list)
row_list = list(zip(res.x, [[] for x in range(len(res.x))]))

for col, row in zip(*np.nonzero(matr)):
    col_dict[col].append(row)
    row_list[row][1].append(col)

del matr
del res

def make_probs(x):
    s = sum(x)
    return [y/s for y in x]

chosen = []

while col_dict:
    
    col_idx, rows = next(iter(col_dict.items()))
    
    chosen_row = np.random.choice(rows, p=make_probs([row_list[x][0] for x in rows]))
    
    for x in row_list[chosen_row][1]:
        col_dict.pop(x, None)
    
    chosen.append(chosen_row+1)

for x in sorted(chosen):
    print(x, sep=' ', end=' ')

# 3 Task: Branch & Bound

In [26]:
def get_bounds(items: np.recarray, max_vol: float) -> float:
    
    if len(items) == 0:
        return 0
    
    res = linprog(c=-items.val, A_ub=items.vol.reshape(1, -1), b_ub=[max_vol], bounds=[0, 1], method='interior-point')
    
    rounded = prob_round(res.x, items, max_vol)
    
    return items.val @ res.x, items.val @ rounded

In [25]:
def prob_round(xs, items,  max_vol):
    
    curr_vol = 0
    ans = np.zeros(xs.shape[0])
    probs = np.random.rand(xs.shape[0])
    
    for i in range(xs.shape[0]):
        if curr_vol + items[i].vol <= max_vol:
            add = xs[i] >= probs[i]
            curr_vol += items[i].vol*add
            ans[i] = add
    
    return ans

## BFS like

In [196]:
import numpy as np
from scipy.optimize import linprog

# Input

def read_data(test=False):
    
    if test:
        total_volume = 31181
        items_str = """4990 1945
        1142 321
        7390 2945
        10372 4136
        3114 1107
        2744 1022
        3102 1101
        7280 2890
        2624 962
        3020 1060
        2310 805
        2078 689
        3926 1513
        9656 3878
        32708 13504
        4830 1865
        2034 667
        4766 1833
        40006 16553"""
        items_list = []
        for x in items_str.split('\n'):
            vol, val = map(float, x.split())
            if vol <= total_volume:
                items_list.append((vol, val))
    else:
        total_volume = float(input())
        num_obj = int(input())

        items_list = []
        for i in range(num_obj):
            vol, val = map(float, input().split())
            if vol <= total_volume:
                items_list.append((vol, val))    
            
    items = np.array(items_list, dtype=np.dtype([('vol', float), ('val', float)])).view(np.recarray)
    return total_volume, items

# Bounds

def bound_max(items: np.recarray, max_vol: float) -> float:
    
    if len(items) == 0:
        return 0
    
    res = linprog(c=-items.val, A_ub=items.vol.reshape(1, -1), b_ub=[max_vol], bounds=[0, 1], method='interior-point')
    
    return items.val @ res.x

def bound_min(items: np.array, max_vol: float) -> float:

    item_scores = map(lambda x: x.val/x.vol, items)
    
    val, vol = 0, 0
    
    for idx, score in sorted(zip(np.arange(len(items)), item_scores), key=lambda x: x[1], reverse=True):
        if vol + items[idx].vol <= max_vol:
            vol += items[idx].vol
            val += items[idx].val
    return val

# B&B

class Route:
    
    def __init__(self, prev, choice: bool, item: np.recarray, minb, maxb):
    
        if prev is None:
            self.steps = []
            self.minb = 0
            self.maxb = 0
            self.vol = 0
            self.val = 0
        else:
            self.steps = prev.steps.copy()
            self.steps.append(choice)
            
            self.vol = prev.vol + item.vol*choice
            self.val = prev.val + item.val*choice
            self.minb = minb
            self.maxb = maxb
    
    def __str__(self) -> str:
        return "steps: {}, vol: {}, val: {}, minb: {}, maxb: {}".format(self.steps, self.vol, self.val, self.minb, self.maxb)
    
    def __repr__(self) -> str:
        return self.__str__()

def branch(routes, item_idx, items, total_volume):
    '''
    This function bounds optimum values for given prefixes on addition of new item.
    '''
    item = items[item_idx]
    scores = []
    
    for route_idx, route in enumerate(routes):
        for choice in [0, 1]:
            
            curr_items = items[item_idx+1:]
            curr_vol = route.vol + item.vol * choice
            curr_val = route.val + item.val * choice
            
            minb = bound_min(curr_items, total_volume-curr_vol)
            maxb = bound_max(curr_items, total_volume-curr_vol)
            
            scores.append( [route_idx, choice, curr_val+minb, curr_val+maxb] )
    
    return scores

def bound(scores, routes, item, total_volume):
    '''
    Delete branches which are defenetely worse
    '''
    possible = filter(lambda x: routes[x[0]].vol+item.vol*x[1] <= total_volume, scores)
    
    max_min = max(possible, key=lambda x: x[2])[2]
    
    return filter(lambda x: x[3] >= max_min and routes[x[0]].vol+item.vol*x[1] <= total_volume, scores)

In [197]:
%%time 

total_volume, items = read_data(test=True)

routes = [Route(None, None, None, None, None)]

for item_idx, item in enumerate(items):
    
    scores = branch(routes, item_idx, items, total_volume)
    
    new_routes = [Route(prev=routes[step[0]], choice=step[1], item=item, minb=step[2], maxb=step[3]) 
                                         for step in bound(scores, routes, item, total_volume)]
    
    routes = new_routes
    
    #for x in routes:
    #    print(x)
    #print('\n')
    #input()

print(int(max(routes, key=lambda x: x.val).val))

12248
CPU times: user 6.64 s, sys: 84.9 ms, total: 6.73 s
Wall time: 6.61 s


## With priority queue

In [207]:
import numpy as np
from scipy.optimize import linprog

def read_data(test=False):
    
    if test:
        total_volume = 31181
        items_str = """4990 1945
        1142 321
        7390 2945
        10372 4136
        3114 1107
        2744 1022
        3102 1101
        7280 2890
        2624 962
        3020 1060
        2310 805
        2078 689
        3926 1513
        9656 3878
        32708 13504
        4830 1865
        2034 667
        4766 1833
        40006 16553"""
        items_list = []
        for x in items_str.split('\n'):
            vol, val = map(float, x.split())
            if vol <= total_volume:
                items_list.append((vol, val))
    else:
        total_volume = float(input())
        num_obj = int(input())

        items_list = []
        for i in range(num_obj):
            vol, val = map(float, input().split())
            if vol <= total_volume:
                items_list.append((vol, val))    
            
    items = np.array(items_list, dtype=np.dtype([('vol', float), ('val', float)])).view(np.recarray)
    return total_volume, items

def bound(items: np.recarray, max_vol: float) -> float:
    
    item_scores = map(lambda x: x.val/x.vol, items)
    
    val, vol = 0, 0
    max_bound = 0
    
    for idx, score in sorted(zip(np.arange(len(items)), item_scores), key=lambda x: x[1], reverse=True):
        if vol + items[idx].vol <= max_vol:
            vol += items[idx].vol
            val += items[idx].val
        elif max_bound == 0:
            max_bound = val + items[idx].val * (total_volume - vol) / items[idx].vol
            
    return val, max_bound

def bound_min_with_path(items: np.array, max_vol: float) -> float:

    item_scores = map(lambda x: x.val/x.vol, items)
    result = np.zeros(items.shape[0])
    
    val, vol = 0, 0
    
    for idx, score in sorted(zip(np.arange(len(items)), item_scores), key=lambda x: x[1], reverse=True):
        if vol + items[idx].vol <= max_vol:
            vol += items[idx].vol
            val += items[idx].val
            result[idx] = 1
            
    return val, result


class Node:
    
    def __init__(self, obj_id, val, vol, parent, items, total_volume): 
        
        self.obj_id = obj_id
        self.val = val
        self.vol = vol
        self.parent = parent
        self.total_volume = total_volume
        self.items = items
        
        minb, maxb = bound(items[obj_id+1:], total_volume-self.vol)
        self.minb = self.val + minb
        self.maxb = self.val + maxb
        
        self.left = None
        self.right = None
        
    def make_children(self):
        
        if self.obj_id+1 >= self.items.shape[0]:
            left = None
        else:
            left = Node(
                obj_id = self.obj_id+1, 
                val = self.val, 
                vol = self.vol, 
                parent = self, 
                items = self.items, 
                total_volume = self.total_volume
            )
        
        if self.obj_id+1 >= self.items.shape[0] or self.vol + self.items[self.obj_id+1].vol > self.total_volume:
            right = None
        else:
            right = Node(
                obj_id = self.obj_id+1, 
                val = self.val+self.items[self.obj_id+1].val,
                vol = self.vol+self.items[self.obj_id+1].vol,
                parent = self,
                items = self.items,
                total_volume = self.total_volume
            )
            
        self.left = left
        self.right = right
        
        return self.left, self.right
    
    def __lt__(self, other):
        return self.minb > other.minb
    
    def __str__(self):
        
        res = []
        
        if self.left:
            res.extend(map(lambda x: '0 '+ x, str(self.left).split('\n')))
        if self.right:
            res.extend(map(lambda x: '1 '+ x, str(self.right).split('\n')))
            
        if not len(res):
            res.append(': minb:{}, maxb: {}'.format(self.minb, self.maxb))
        return '\n'.join(res)

def construct_branch_from_solution(solution, items, total_volume):
    
    root = Node(
        obj_id=-1,
        val=0,
        vol=0,
        parent=None,
        items = items,
        total_volume = total_volume
    )
    curr_n = root
    
    pq = []    
    for choice in solution:
        left, right = curr_n.make_children()
        
        if choice:
            curr_n = right
            if left:
                pq.append(left)
        else:
            curr_n = left
            if right:
                pq.append(right)
                
    return root, pq

In [210]:
%%time
total_volume, items = read_data(test=True)

best_bound, solution = bound_min_with_path(items, total_volume)

root, pq = construct_branch_from_solution(solution, items, total_volume)

while len(pq):
    
    node = pq.pop()
    
    if node.maxb < best_bound:
        continue
    
    if node.minb > best_bound:
        best_bound = node.minb
    
    left, right = node.make_children()
    
    if left:
        pq.append(left)
    if right:
        pq.append(right)
        
    #print(root)
    #print('best: {}', best_bound)
    #input()

print(int(best_bound))

12248
CPU times: user 3.93 s, sys: 79 ms, total: 4.01 s
Wall time: 3.78 s


In [1]:
# other solution

In [ ]:
from scipy.optimize import linprog
import numpy as np
import sys

V = int(input())
N = int(input())
volumes = [None] * N
prices = [None] * N

for i in range(N):
    raw_in = input().split()
    volumes[i] = int(raw_in[0])
    prices[i]  = int(raw_in[1])

# Sort input greedy
items = sorted(zip(volumes, prices), key=lambda x: -x[1]/x[0])

volumes = np.array([v for v, _ in items])
prices  = np.array([p for _, p in items])

def get_opt(V=V, volumes=volumes, prices=prices):
    sum_price = 0

    for v, p in zip(volumes, prices):
        if V < v:
            return sum_price + p * V/v

        V -= v
        sum_price += p

    return sum_price

sys.setrecursionlimit(10**6)

# Best here is theoretical best (best - current price) for the current part of the tree
def try_take(V=V, volumes=volumes, prices=prices, best=0):
    # No items left
    if len(volumes) == 0:
        return 0
    
    # Optimization for leaves
    if len(volumes) == 1:
        if volumes[0] < V:
            return prices[0]
        else:
            return 0

    if get_opt(V, volumes, prices) <= best:
        return 0

    # Try to add
    if volumes[0] <= V:        
        taken_best = try_take(V - volumes[0], volumes[1:], prices[1:], best - prices[0])
        best = max(taken_best + prices[0], best)
    
    # If we don't add
    not_taken_best = try_take(V, volumes[1:], prices[1:], best)
    best = max(best, not_taken_best)
    return best

print(int(try_take()))